# Model for a magnetic skyrmion

A linear model to describe the vector field of spins (classical model) of a magnetic skyrmion [1] is given by

$$\vec{s}=\zeta\sin(\Theta)\cos(\chi \Psi + \phi) \hat{x} + \zeta\sin(\Theta)\sin(\chi \Psi + \phi) \hat{y} - \zeta\cos(\theta) \hat{z}$$

with the functions $\Theta$ and $\Psi$ described in cylindrical coordinates. Notice that the spin vector in this equations is normalized. A linear approximation [1] for the $\Theta$ function is given by

$$\Theta = \frac{\pi}{r_\mathrm{sk}} r$$
$$r=\sqrt{x^2 + y^2}$$

Outside $r_{\mathrm{sk}}$, the background of spins is given by $(0, 0, \zeta)$, which is opposite to the orientation of the spin at the skyrmion center.

The skyrmion profile can be modified by its helicity, chirality and vorticity. The chirality specifies the sense of rotation of the magnetic configuration. The vorticity can have values -1 or 1, and the helicity is specified with values in the $[0, \pi]$ range. 

In this notebook, we use this model to plot the spin field in a triangular lattice of spins. 

[1] Beg *et al*, Scientific Reports **5**, 17137 (2015) https://www.nature.com/articles/srep17137


## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.collections import PolyCollection
from ipywidgets import HBox, VBox, FloatSlider, IntSlider, link
from IPython.display import display, clear_output

In [2]:
%matplotlib widget

In [3]:
plt.clf()
plt.ion()

FigureCanvasNbAgg()

## Functions

In [4]:
def init_coordinates(nx, ny, radius):
    """
    Generates the coordinates of a nx * ny triangular lattice of points
    arranged in a square geometry
    
    The distance between consecutive points are specified by the radius of the
    hexagons defined by the triangular lattice
    """
    n = nx * ny
    dy = np.sqrt(3) * radius
    dx = 2.0 * radius
    h = dx * 2. / np.sqrt(3)
    coordinates = np.zeros((n, 3))
    for j in range(ny):
        for i in range(nx):
            index = get_index(nx, ny, i, j)
            # For a square alignment, the hexagons will
            # always be in the same x-position for even numbered rows (j)
            # x(i=0) = dx * 0.5
            # if alignment == 'square':
            if j % 2 == 0:
                sign = 1
            else:
                sign = 0

            r = (sign * dx / 2.0 + i * dx + dx / 2.0,
                 j * dy + h / 2.0,
                 0)

            coordinates[index] = r
    return coordinates

def get_index(nx, ny, i, j):
    """
    Returns the index for the cell with ordinals i, j
    or False if that cell would be out of bounds.

    """
    if i < 0 or j < 0 or i >= nx or j >= ny:
        return -1
    return i + j * nx

def hexagon_corners(x, y, radius):
    """
    Returns the coordinates of the corners of the hexagonal with
    center at `x`, `y` and radius `radius`.

    """
    angle_deg = [60 * i + 30 for i in range(6)]
    angle_rad = [a * np.pi / 180 for a in angle_deg]
    return [(x + radius * np.cos(theta),
             y + radius * np.sin(theta),
             0) for theta in angle_rad]

def init_corners(coordinates, nx, ny, radius):
    """
    Generates the corners of the hexagons defined by the triangular lattice
    The lattice coordinates are used as input
    """
    dy = np.sqrt(3) * radius
    dx = 2.0 * radius
    h = dx * 2. / np.sqrt(3)

    corners = []
    for j in range(ny):
        for i in range(nx):
            index = get_index(nx, ny, i, j)
            x, y = c[index][0], c[index][1]
            # self.radius is the inradius while self.h/2  is the circumradius
            corners.append(hexagon_corners(x, y, h * 0.5))

    corners = np.array(corners)
    corners = corners[:, :, :2]
    
    return corners

def s_skyrmion(r, r_sk, center=(0, 0), helicity=(0.5 * np.pi), vorticity=1, chirality=1):
    """
    A linear model for the spin field of a magnetic skyrmion
    
    Given a coordinate r, and skyrmion radius r_sk, produce the spin field:
    
    m = chirality * sin theta cos( psi + helicity) X
        chirality * sin theta sin( psi + helicity) Y
        -chirality * cos theta Z
        
    If the position is outside r_sk, the function returns (0, 0, chirality),
    which is opposite to the skyrmion center
    """
    x, y = r[0] - center[0], r[1] - center[1]
    rho = np.sqrt(x ** 2 + y ** 2)
    psi = vorticity * np.arctan2(y, x) + helicity
    k = np.pi / r_sk
    
    if rho <= r_sk:
        return np.array([chirality * np.sin(k * rho) * np.cos(psi),
                         chirality * np.sin(k * rho) * np.sin(psi),
                         -chirality * np.cos(k * rho)]
                        )
    else:
        return np.array([0, 0, chirality])

# Vectorize the spin field generator
# Might not be necessary, we could also use np.apply_along_axis
s_skyrmion_v = np.vectorize(s_skyrmion, 
                            signature='(3)->(3)',  # signature of pyfunc!
                            otypes=[np.float],
                            excluded=['r_sk', 'center', 'helicity', 'vorticity', 'chirality'])

def get_spins(coordinates, r_sk, center=(0, 0), helicity=0, vorticity=1, chirality=1):
    """
    Returns the spin field with a skyrmion, using the coordinates from a triangular lattice
    Skyrmion parameters can be passed as arguments
    """
    spin = s_skyrmion_v(coordinates, r_sk=r_sk, center=center, helicity=helicity, vorticity=vorticity, chirality=chirality)
    spin_n = np.sqrt(spin[:, 0] ** 2 + spin[:, 1] ** 2 + spin[:, 2] ** 2)
    spin[spin_n > 0] /= spin_n[spin_n > 0][:, np.newaxis]
    return spin

## Widget

The plot shows the in plane component of the atoms, in the $xy$ plane. The colours are from the out of plane components of the spin, $s_z$

In [5]:
nx, ny, radius = 30, 30, 1

# Generate the coordinates of the triangular lattice of spins
c = init_coordinates(nx, ny, radius)
cenx = (c[:, 0].max() - c[:, 0].min()) * 0.5 + c[:, 0].min()
ceny = (c[:, 1].max() - c[:, 1].min()) * 0.5 + c[:, 1].min()

# Get the corners of the hexagons defined by the triangular lattice
corners = init_corners(c, nx, ny, radius)

# Get the spin field with a skyrmion
spin = get_spins(c, r_sk=14, center=(cenx, ceny))

In [6]:
plt.close('all')

In [7]:
z = np.random.random(corners.shape[0])

fig, ax = plt.subplots(figsize=(6, 5))

hel_slider = FloatSlider(orientation='horizontal',
                         value=0.0, min=0.0, max=np.pi, step=np.pi/10,
                         description='Helicity ' + r'$\phi$')

vor_slider = IntSlider(orientation='horizontal',
                       value=1, min=-1, max=1, step=2,
                       description='Vorticity ' + r'$\xi$')

chi_slider = IntSlider(orientation='horizontal',
                       value=1, min=-1, max=1, step=2,
                       description='Chirality ' + r'$\zeta$')

rad_slider = IntSlider(orientation='horizontal',
                       value=14, min=2, max=25, step=1,
                       description='Sk radius')

# Make the collection of hexagons and add it to the plot.
# We draw hexagons at every spin position
coll = PolyCollection(corners, 
                      # array=z,
                      array=spin[:, 2],
                      cmap=mpl.cm.RdYlBu,
                      edgecolors='none',
                      alpha=0.3
                      )
ax.add_collection(coll)
ax.autoscale_view()

# Plot the directions of the spins in the XY plane
q = ax.quiver(c[:, 0], c[:, 1], spin[:, 0], spin[:, 1],
              spin[:, 2],
              width=0.004, headwidth=4, headlength=5, edgecolor='k', lw=0.2,
              scale=30, pivot='mid',
              cmap='RdYlBu'
              )

ax.set_title('Skyrmion model' + '\n' +
            r'$\vec{m}=\zeta\sin(\Theta)\cos(\chi \Psi + \phi) \hat{x} + \zeta\sin(\Theta)\sin(\chi \Psi + \phi) \hat{y} - \zeta\cos(\theta) \hat{z}$')

# Colorbar
box = ax.get_position()
axCb = plt.axes([box.x0 + 0.015, 0.2, box.width * 0.2, 0.02])
cb = mpl.colorbar.ColorbarBase(axCb, plt.cm.RdYlBu, orientation="horizontal",
                               ticks=[-1, 0, 1], 
                               norm=mpl.colors.Normalize(vmin=-1, vmax=1))
cb.set_label(r'$s_{z}$', rotation=0, y=.5, labelpad=-0, fontsize=14)
# axCb.yaxis.set_ticks_position('left')

clear_output()

# Widget --------------------------------------------------------------------------------
# Here we tune the skyrmion profile and update the plot accordingly

def update(change, param='chirality'):
    if param=='chirality':
        spin = get_spins(c, r_sk=rad_slider.value, center=(cenx, ceny), 
                         helicity=hel_slider.value, vorticity=vor_slider.value, chirality=change.new)
        
        # coll.set_array(-np/ones((spin.shape[0] // 3)))
        coll.set_array(spin[:, 2])
        
    elif param=='vorticity':
        spin = get_spins(c, r_sk=rad_slider.value, center=(cenx, ceny),
                         helicity=hel_slider.value, vorticity=change.new, chirality=chi_slider.value)    
    elif param=='helicity':
        spin = get_spins(c, r_sk=rad_slider.value, center=(cenx, ceny),
                         helicity=change.new, vorticity=vor_slider.value, chirality=chi_slider.value)    
    elif param=='radius':
        spin = get_spins(c, r_sk=change.new, center=(cenx, ceny),
                         helicity=hel_slider.value, vorticity=vor_slider.value, chirality=chi_slider.value)
        
    q.set_UVC(spin[:, 0], spin[:, 1], spin[:, 2])
    # q.update()    
    fig.canvas.draw()
    fig.canvas.flush_events()
    # fig.tight_layout()
    
hel_slider.observe(lambda c: update(c, param='helicity'), names='value')
vor_slider.observe(lambda c: update(c, param='vorticity'), names='value')
chi_slider.observe(lambda c: update(c, param='chirality'), names='value')
rad_slider.observe(lambda c: update(c, param='radius'), names='value')

VBox([hel_slider, vor_slider, chi_slider, rad_slider, fig.canvas])